In [1]:
import numpy as np

In [2]:
class CurrentBoard:
    def __init__(self, setup_of_board = "         "):
        self.X = "X"
        self.O = "O"
        self.space = " "
        self.board_size = 9
    
        if len(setup_of_board) != self.board_size:
            print("Invalid Setup Error, Length expected: 9, Length provided: {}".format(len(setup_of_board)))
        else:
            self.board = setup_of_board
        self.outcome = self.game_won()
        self.game_board = self.construct_display_string()

    
    def display(self, board_string = ""):
        if board_string == "":
            c = self.construct_display_string()
        else:
            c = board_string
        print("{}|{}|{}".format(c[0], c[1], c[2]))
        print("------")
        print("{}|{}|{}".format(c[3], c[4], c[5]))
        print("------")
        print("{}|{}|{}".format(c[6], c[7], c[8]))
    
    def construct_display_string(self):
        hold = ""
        for i in range(self.board_size):
            if self.board[i] == self.space:
                hold += str(i)
            else:
                hold += self.board[i]
        return hold
            
    def equal3(self, a, b, c):
        return (self.board[a] == self.board[b]) and (self.board[a] == self.board[c]) and (self.board[a] != self.space)
            
    def game_won(self):
        # returns x if x wins, o if o wins, " " if no result or d if draw
        if self.equal3(0, 1, 2) or self.equal3(0, 3, 6) or self.equal3(0, 4, 8):
            return self.board[0]
        elif self.equal3(1, 4, 7):
            return self.board[1]
        elif self.equal3(2, 5, 8) or self.equal3(2, 4, 6):
            return self.board[2]
        elif self.equal3(3, 4, 5):
            return self.board[3]
        elif self.equal3(6, 7, 8):
            return self.board[6]
        else:
            return self.space
        
    def other(self, character):
        if character == self.X:
            return self.O
        if character == self.O:
            return self.X
        print("Error character not recognised")
        return self.space
    
    def all_possible_moves_for(self, choosing_this_character):
        #returns (as GameStates) a list of all possible moves
        list_of_all_moves = []
        if self.game_won() != " ":
            print("Game has ended, no possible moves")
        else:
            for i in range(self.board_size):
                if(self.board[i] == self.space):
                    list_of_all_moves.append(CurrentBoard(self.board[:i]+choosing_this_character+self.board[i+1:]))
        return list_of_all_moves

In [3]:
class SearchTreeNode():
    def __init__(self, just_after_choosing_character, current_game = None, Ply_Depth = 0):
        self.children = []
        self.ply_depth = Ply_Depth
        self.is_node_value_assigned = False
        self.node_value = 0
        self.choosing_character = just_after_choosing_character
        
        if current_game == None:
            self.game_at_this_node = CurrentBoard()
        else:
            self.game_at_this_node = current_game
            
        self.generate_children()
        
    def generate_children(self):
        if self.game_at_this_node.game_won() != self.game_at_this_node.space:
            #This is a leaf, apply utility value function
            if (self.ply_depth) % 2 == 1:
                self.node_value = 1
            else:
                self.node_value = -1
            self.is_node_value_assigned = True
            
        else:
            all_moves = self.game_at_this_node.all_possible_moves_for(self.game_at_this_node.other(self.choosing_character))
            if (len(all_moves) == 0):
                self.node_value = 0
                self.is_node_value_assigned = True
            else:
                for new_game in all_moves:
                    self.children.append(SearchTreeNode(new_game.other(self.choosing_character), new_game, self.ply_depth+1))
        
    def get_node_value(self):
        if not self.is_node_value_assigned:
            self.is_node_value_assigned = True
            self.children = sorted(self.children, key = lambda x: x.get_node_value())
            
            if (self.ply_depth % 2) == 0: # Maximising Layer
                self.node_value = self.children[-1].node_value
            else: # Minimising Layer
                self.node_value = self.children[0].node_value
            
        return self.node_value


In [5]:
def play_tic_tac_toe():
    current_board = None
    player_choosing = ""
    computer_choosing = ""
    choice = input("Do you want to go first? y/n ")
    player_going_first = (choice == "y" or choice == "Y")
    choice = input("Do you want to play as X? y/n ")
    if choice == "y" or choice == "Y":
        player_choosing = "X"
        computer_choosing = "O"
    else:
        player_choosing = "O"
        computer_choosing = "X"
    if player_going_first:    
        current_board = CurrentBoard()
    else:
        #Choose a random move
        current_board = CurrentBoard()
        ind = np.random.randint(current_board.board_size)
        current_board = CurrentBoard(current_board.board[:ind] + computer_choosing + current_board.board[ind+1:])
        
    for i in range(9):
        current_board.display()
        index_as_string = input("Choose your move ")
        ind = int(index_as_string)
        current_board = CurrentBoard(current_board.board[:ind] + player_choosing + current_board.board[ind+1:])
        if current_board.outcome != current_board.space:
            print("I can't believe you've won!!")
            current_board.display()
            break
        elif len(current_board.all_possible_moves_for(computer_choosing)) == 0:
            print("this is a draw")
            current_board.display()
            break
        search_tree = SearchTreeNode(player_choosing, current_board)
        search_tree.get_node_value()
        current_board = search_tree.children[-1].game_at_this_node
        if current_board.outcome != current_board.space:
            print("Haha you lost loser")
            current_board.display()
            break
        elif len(current_board.all_possible_moves_for(player_choosing)) == 0:
            print("issa draw")
            current_board.display()
            break
        
    return player_going_first, player_choosing, current_board

In [6]:
a,b,c = play_tic_tac_toe()

Do you want to go first? y/n y
Do you want to play as X? y/n o
0|1|2
------
3|4|5
------
6|7|8
Choose your move 4
0|1|2
------
3|O|5
------
6|7|X
Choose your move 7
0|X|2
------
3|O|5
------
6|O|X
Choose your move 5
0|X|2
------
X|O|O
------
6|O|X
Choose your move 0
O|X|2
------
X|O|O
------
X|O|X
Choose your move 2
this is a draw
O|X|O
------
X|O|O
------
X|O|X


In [ ]:
Game = SearchTreeNode("X", CurrentBoard(" X  XO   "))

In [ ]:
Game.game_at_this_node.display_gameboard()

In [ ]:
test = CurrentBoard()

In [ ]:
test.display()

In [ ]:
test.display()

In [ ]:
Game.get_node_value()

In [ ]:
for b in Game.children[3].children[0].children:
    print()
    b.game_at_this_node.display()
    print(b.node_value)
    print()

In [ ]:
Game.children[3].children[1].children[3].game_at_this_node.display()

In [ ]:
Game.children[3].children[1].is_node_value_assigned

In [ ]:
cb = CurrentBoard("OX  O XX")

In [ ]:
cb.display()

In [ ]:
all_moves = cb.all_possible_moves_for("O")

In [ ]:
all_moves

In [ ]:
for b in all_moves:
    print()
    b.display()